In [0]:


import os, glob
import pandas as pd

DATA_DIR = os.path.join('..', 'tick_data', 'date=2024-03-01')
REQ_COLS = ['datetime', 'currency_pair', 'bid', 'ask', 'volume']

files = {
    'A': sorted(glob.glob(os.path.join(DATA_DIR, 'sample_fx_data_A.csv.gz'))),
    'B': sorted(glob.glob(os.path.join(DATA_DIR, 'sample_fx_data_B.csv.gz'))),
    'C': sorted(glob.glob(os.path.join(DATA_DIR, 'sample_fx_data_C.csv.gz'))),
}

def check_schema(path, req_cols=REQ_COLS):
    """
    Read only the header of a .csv.gz file and validate required columns.
    """
    hdr = pd.read_csv(path, nrows=0, compression='infer')
    cols = list(hdr.columns)
    missing = [c for c in req_cols if c not in cols]
    if missing:
        raise ValueError(f"{path}: missing required columns {missing}. Found: {cols}")
    return True

# --- Run schema checks ---
for prov, paths in files.items():
    if not paths:
        print(f"[WARN] No file found for provider {prov}")
        continue
    for p in paths:
        check_schema(p)
        print(f"[OK] Schema valid for provider {prov}: {os.path.basename(p)}")



In [0]:
import numpy as np

VALID_CURRENCY_PAIRS = {
    "USDJPY",
    "EURJPY",
    "AUDJPY"
}


# Load & validate + enrich
def load(paths, provider):
    parts = []
    for p in paths:
        df = pd.read_csv(p)

        # ✅ Schema check
        miss = set(REQ_COLS) - set(df.columns)
        if miss:
            raise ValueError(f"{provider}: missing columns {miss} in {p}")

        df = df[REQ_COLS].copy()
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

        # ✅ Currency pair validation
        invalid_ccy = set(df['currency_pair'].dropna().unique()) - VALID_CURRENCY_PAIRS
        if invalid_ccy:
            raise ValueError(
                f"{provider}: invalid currency_pair(s) {invalid_ccy} in {p}"
            )

        # ✅ Add provider
        df['provider'] = provider

        # ✅ Enrichment (now added back)
        df['mid'] = (df['bid'] + df['ask']) / 2.0
        df['spread'] = df['ask'] - df['bid']
        df['rel_spread_bps'] = (
            (df['spread'] / df['mid']) * 1e4
        ).replace([np.inf, -np.inf], np.nan)
        df['invalid_price'] = df['ask'] <= df['bid']

        parts.append(df)

    return pd.concat(parts, ignore_index=True)

raw = pd.concat([load(paths, prov) for prov, paths in files.items()], ignore_index=True
)
print(raw)





In [0]:

# Provider-level summary
basic = raw.groupby('provider').agg(
    rows=('provider','size'),
    pairs=('currency_pair', pd.Series.nunique),
    start=('datetime','min'),
    end=('datetime','max'),
    invalid_price_rate=('invalid_price','mean'),
    median_spread=('spread','median'),
    median_rel_bps=('rel_spread_bps','median'),
    p95_rel_bps=('rel_spread_bps', lambda s: np.percentile(s.dropna(),95)),
).reset_index()

# Continuity by pair
raw_sorted = raw.sort_values(['provider','currency_pair','datetime'])
gap_rows = []
for (prov, pair), g in raw_sorted.groupby(['provider','currency_pair']):
    gap = g['datetime'].diff().dt.total_seconds().dropna()
    if gap.size:
        gap_gt1 = float(np.mean(gap>1.0))
        med_gap = float(np.median(gap))
    else:
        gap_gt1 = np.nan
        med_gap = np.nan
    gap_rows.append({'provider':prov,'currency_pair':pair,'pct_gaps_gt_1s':gap_gt1,'median_gap_s':med_gap})

gaps_df = pd.DataFrame(gap_rows)
cont = gaps_df.groupby('provider').agg(
    avg_pct_gaps_gt_1s=('pct_gaps_gt_1s','mean'),
    median_gap_s_over_pairs=('median_gap_s','median')
).reset_index()

raw['dup'] = raw.duplicated(['provider','currency_pair','datetime','bid','ask','volume'])
dups = raw.groupby('provider')['dup'].mean().reset_index(name='duplicate_rate')

summary = basic.merge(cont, on='provider', how='left').merge(dups, on='provider', how='left')
summary


In [0]:

# Per-pair spread detail
pair_spread = raw.groupby(['provider','currency_pair']).agg(
    rows=('provider','size'),
    median_rel_bps=('rel_spread_bps','median'),
    p95_rel_bps=('rel_spread_bps', lambda s: np.percentile(s.dropna(),95)),
    invalid_price_rate=('invalid_price','mean'),
).reset_index()
pair_spread


In [0]:
import matplotlib.pyplot as plt

# Basic visuals
fig, axes = plt.subplots(1,2, figsize=(11,4))
raw.boxplot(column='spread', by='provider', ax=axes[0])
axes[0].set_title('Absolute Spread by Provider')
axes[0].set_xlabel('Provider'); axes[0].set_ylabel('Spread')

raw.boxplot(column='rel_spread_bps', by='provider', ax=axes[1])
axes[1].set_title('Relative Spread (bps) by Provider')
axes[1].set_xlabel('Provider'); axes[1].set_ylabel('bps')
plt.suptitle(''); plt.tight_layout(); plt.show()


In [0]:

from pathlib import Path

OUTPUT_HTML = Path("provider_analysis.html")

with open(OUTPUT_HTML, "w") as f:
    f.write("""
<html>
<head>
    <title>FX Data Provider Analysis</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; }
        h1, h2, h3 { color: #2c3e50; }
        table { border-collapse: collapse; margin: 20px 0; width: 100%; }
        th, td { border: 1px solid #ccc; padding: 8px; text-align: right; }
        th { background-color: #f5f5f5; }
        td:first-child, th:first-child { text-align: left; }
        section { margin-bottom: 40px; }
    </style>
</head>
<body>

<h1>FX Tick Data Provider Analysis</h1>

<section>
<h2>Overview</h2>
<p>
This report evaluates FX tick data quality across three providers (A, B, C) using:
<ul>
<li>Schema and currency validation</li>
<li>Market sanity (crossed prices)</li>
<li>Microstructure quality (spreads)</li>
<li>Continuity and duplication</li>
</ul>
</p>
</section>

<section>
<h2>Provider-Level Summary</h2>
""")

    # Provider summary table
    f.write(summary.to_html(index=False, float_format="%.4f"))

    f.write("""
</section>

<section>
<h2>Per-Currency Pair Spread Analysis</h2>
""")

    # Pair-level stats
    f.write(pair_spread.to_html(index=False, float_format="%.4f"))

    f.write("""
</section>

<section>
<h2>Key Observations</h2>
<ul>
<li>Lower relative spreads indicate better pricing quality.</li>
<li>Invalid price rate highlights crossed or locked markets.</li>
<li>Gaps &gt; 1 second indicate weaker tick continuity.</li>
<li>Duplicate rate shows potential replayed ticks.</li>
</ul>
</section>

<section>
<h2>Conclusion</h2>
<p>
Based on pricing sanity, spread tightness, and continuity metrics, Provider A
demonstrates superior data quality in the sampled dataset.
</p>
</section>

</body>
</html>
""")

print(f"✅ HTML report generated: {OUTPUT_HTML.absolute()}")


In [0]:

plt.figure(figsize=(11,4))
raw.boxplot(column='spread', by='provider')
plt.title('Absolute Spread by Provider')
plt.suptitle('')
plt.tight_layout()
plt.savefig("absolute_spread.png")
plt.close()

plt.figure(figsize=(11,4))
raw.boxplot(column='rel_spread_bps', by='provider')
plt.title('Relative Spread (bps) by Provider')
plt.suptitle('')
plt.tight_layout()
plt.savefig("relative_spread.png")
plt.close()
